# 【課題】腫瘍の良性／悪性を予測（分類）しよう

## データを読み込む

`breast_cancer_wisconsin_data.csv` を読み込みましょう。

In [17]:
# Pandasを使ってbreast_cancer_wisconsin_data.csvを読み込む（変更しないでください）
import pandas as pd
bc_data = pd.read_csv("breast_cancer_wisconsin_data.csv")

読み込んだデータについて確認します。

In [18]:
# 読み込んだデータが何行何列かをshapeで確認する（変更しないでください）
print(bc_data.shape)

(569, 33)


In [26]:
# 読み込んだデータの先頭5行を確認する（変更しないでください）
bc_data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [27]:
# 読み込んだデータの先頭の1行だけを確認する（変更しないでください）
bc_data.iloc[0, :]

id                           842302
diagnosis                         M
radius_mean                   17.99
texture_mean                  10.38
perimeter_mean                122.8
area_mean                    1001.0
smoothness_mean              0.1184
compactness_mean             0.2776
concavity_mean               0.3001
concave points_mean          0.1471
symmetry_mean                0.2419
fractal_dimension_mean      0.07871
radius_se                     1.095
texture_se                   0.9053
perimeter_se                  8.589
area_se                       153.4
smoothness_se              0.006399
compactness_se              0.04904
concavity_se                0.05373
concave points_se           0.01587
symmetry_se                 0.03003
fractal_dimension_se       0.006193
radius_worst                  25.38
texture_worst                 17.33
perimeter_worst               184.6
area_worst                   2019.0
smoothness_worst             0.1622
compactness_worst           

このCSVファイルには30以上の列があります。主要な列のみ、以下に概要を記載します。

- id：連番
- diagnosis："B"か"M"の文字が格納されている（"B"：良性、"M"：悪性）
- radius_mean：中心から外周までの平均距離（半径）
- texture_mean：グレースケール（色の濃さ）の平均値
- perimeter_mean：外周の平均の長さ

今回は `radius_mean`（半径）と `perimeter_mean`（外周の長さ）を説明変数、`diagnosis`（良性か悪性か）を目的変数として分類を行ないます。

データの前処理や分割を行なう前に、スライスを使って、`radius_mean`列と`perimeter_mean`列を変数`X`、`diagnosis` 列のみを変数 `y` に格納してください。また、PandasでCSVを読み込んだため、データはDataFrame形式となっています。そのため、データはndarray形式に変換してください。

In [28]:
# スライスとndarrayへの変換をして X と y を作成する
# （以下にコードを書いてください）
# X に radius_mean と perimeter_mean を格納（ndarray形式に変換）
X = bc_data.loc[:, ["radius_mean", "perimeter_mean"]].values

# y に diagnosis を格納（ndarray形式に変換）
y = bc_data.loc[:, "diagnosis"].values

# 確認
print("X の shape:", X.shape)
print("y の shape:", y.shape)
print("X の先頭5行:\n", X[:5])
print("y の先頭5行:\n", y[:5])

X の shape: (569, 2)
y の shape: (569,)
X の先頭5行:
 [[ 17.99 122.8 ]
 [ 20.57 132.9 ]
 [ 19.69 130.  ]
 [ 11.42  77.58]
 [ 20.29 135.1 ]]
y の先頭5行:
 ['M' 'M' 'M' 'M' 'M']


## データの前処理を行なう

さて、`y` には "B" もしくは "M" という文字データが入っています。数値化した方がコンピュータは学習しやすくなるので、データの前処理の1つ「カテゴリ値の数値化」を実行しましょう。カテゴリ値の数値化をするには `LabelEncoder` というものを利用します。

[sklearn.preprocessing.LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html)

※ `LabelEncoder` は、レッスン本文では登場しませんでしたので、以下のコードをそのまま実行すれば大丈夫です。このコードは、上記の公式ドキュメントの内容に沿って記述されています。**なお、このとき `y` は、NumPyのndarray形式に変換されている必要があります。**

In [29]:
# カテゴリ値の数値化
# （変更しないでください）
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(["B", "M"])    # 良性：0, 悪性：1
y = le.transform(y)

# 数値化した状態を確認してみる（変更しないでください）
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,

## データを訓練データとテストデータに分ける

`X` および `y` を訓練データとテストデータに分けましょう。その際、訓練データ8割、テストデータ2割としてください。また、並び順がランダムになるよう、分割してください。

In [30]:
# 訓練データ8割、テストデータ2割に分割する
# （以下にコードを書いてください）
from sklearn.model_selection import train_test_split

# 訓練データ8割、テストデータ2割に分割
# random_state を固定すると実行ごとに結果が同じになる
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

# 確認
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (455, 2)
X_test shape: (114, 2)
y_train shape: (455,)
y_test shape: (114,)


## モデルを作成して訓練する

ここでは、レッスン本文でも利用した「サポートベクターマシン」（`SVC`）の線形分類モデルを使います。また、作成したモデルに訓練データを渡して、学習を行ないます。

In [31]:
# SVCの線形分類モデルを作成する
# （以下にコードを書いてください）
from sklearn.svm import SVC

# SVCの線形分類モデルを作成
model = SVC(kernel="linear", random_state=42)

# 訓練データを使って訓練を行なう
# （以下にコードを書いてください）
model.fit(X_train, y_train)

# 訓練後のモデルを確認（係数や切片など）
print("係数:", model.coef_)
print("切片:", model.intercept_)


係数: [[-2.39451674  0.43524777]]
切片: [-6.55929357]


## 期待する性能が出たかを評価する

分類を行ないましょう。

この分類結果を用いて、モデルの評価を行ないます。

まずは、予測値と実際の値をそのまま表示してください。

In [32]:
# 予測値を表示する
# （以下にコードを書いてください）
# テストデータを使って分類を実行（予測）
y_pred = model.predict(X_test)

# 予測値と実際の値を表示
print("予測値:", y_pred)

予測値: [0 1 1 0 0 1 1 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 0 0 0 1 1 0 0 0 1 0 1
 0 0 0 0 0 0 1 0 0 1 1 1 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 1 1 0 0 0 1 0 0 1
 0 0 1]


In [14]:
# 実際の値を表示する
# （以下にコードを書いてください）
print("実際の値:", y_test)

実際の値: [0 1 1 0 0 1 1 1 0 0 0 1 0 1 0 1 0 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1
 0 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 1 1 0 0 0 1 1 0 0 1 1 0 1
 0 0 0 1 0 0 1 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 0 1 1 0 1 1 0 0 0 1 0 0 1
 0 0 1]


予測の精度を混同行列で示しましょう。

In [15]:
# 混同行列で集計結果を表示する
# （以下にコードを書いてください）
from sklearn.metrics import confusion_matrix

# 混同行列で集計結果を表示する
cm = confusion_matrix(y_test, y_pred)
print("混同行列:\n", cm)

混同行列:
 [[70  1]
 [ 8 35]]


最後に `classification_report()` を実行し、結果を表示してください。

In [16]:
# classification_report()の実行結果を表示する
# （以下にコードを書いてください）
from sklearn.metrics import classification_report

# classification_report() の実行結果を表示する
report = classification_report(y_test, y_pred, target_names=["良性(B)", "悪性(M)"])
print(report)

              precision    recall  f1-score   support

       良性(B)       0.90      0.99      0.94        71
       悪性(M)       0.97      0.81      0.89        43

    accuracy                           0.92       114
   macro avg       0.93      0.90      0.91       114
weighted avg       0.93      0.92      0.92       114

